In [1]:
# 引用套件
import boto3

# 建立 local dynamodb 的連線
dynamodb = boto3.resource(
    'dynamodb',
    endpoint_url = 'http://cc104.dynamodb.local:8000',
    region_name = 'dummy_region',
    aws_access_key_id = 'dummy_access_key',
    aws_secret_access_key = 'dummy_secret_key',
    verify = False )
# 使用dynamodb
#dynamodb = boto3.resource('dynamodb')

In [ ]:
# Create the DynamoDB table.
# table name users
table = dynamodb.create_table(
    TableName='users',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'last_name',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'last_name',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='users')

# Print out some data about the table.
print(table.item_count)

In [3]:
# Using Exiting DynamoDB table
table = dynamodb.Table('users')
print(table.creation_date_time)

2019-01-12 07:23:43.860000+00:00


In [4]:
# Creating a New Item
table.put_item(
   Item={
        'username': 'janedoe',
        'first_name': 'Jane',
        'last_name': 'Doe',
        'age': 25,
        'account_type': 'standard_user',
    }
)

{'ResponseMetadata': {'RequestId': '5a822bc5-d081-4700-b8c0-20dd48421ccb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': '5a822bc5-d081-4700-b8c0-20dd48421ccb',
   'content-length': '2',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [5]:
# Getting an Item
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

{'last_name': 'Doe', 'account_type': 'standard_user', 'first_name': 'Jane', 'age': Decimal('25'), 'username': 'janedoe'}


In [6]:
# Updating Item
table.update_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    },
    UpdateExpression='SET age = :val1',
    ExpressionAttributeValues={
        ':val1': 26
    } 
)

# Getting an Item after Updating
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

{'last_name': 'Doe', 'account_type': 'standard_user', 'first_name': 'Jane', 'age': Decimal('26'), 'username': 'janedoe'}


In [ ]:
# Deleting Item
table.delete_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)

In [9]:
# Querying and Scanning
from boto3.dynamodb.conditions import Key, Attr

# Querying
# Key
response = table.query(
    KeyConditionExpression=Key('username').eq('johndoe')
)
items = response['Items']
print(items)


[]


In [8]:
# Scanning
# Attribute
response = table.scan(
    FilterExpression=Attr('age').lt(27)
)
items = response['Items']
print(items)

[{'last_name': 'Doe', 'account_type': 'standard_user', 'first_name': 'Jane', 'age': Decimal('26'), 'username': 'janedoe'}]


In [10]:
# with logical operators: & (and), | (or), and ~ (not)
response = table.scan(
    FilterExpression=Attr('first_name').begins_with('J') & Attr('account_type').eq('super_user')
)
items = response['Items']
print(items)

[]


In [ ]:
# Deleting table
table.delete()